# Ensemble of ensembles - model stacking

* **Ensemble with different types of classifiers**:
  * Different types of classifiers (E.g., logistic regression, decision trees, random forest, etc.) are fitted on the same training data
  * Results are combined based on either
    * majority voting (classification) or
    * average (regression)
  

* **Ensemble with a single type of classifier**:
  * Bootstrap samples are drawn from training data
  * With each bootstrap sample, model (E.g., Individual model may be decision trees, random forest, etc.) will be fitted
  * All the results are combined to create an ensemble.
  * Suitabe for highly flexible models that is prone to overfitting / high variance.

***

## Combining Method

* **Majority voting or average**:
  * Classification: Largest number of votes (mode)
  * Regression problems: Average (mean).
  
  
* **Method of application of meta-classifiers on outcomes**:
  * Binary outcomes: 0 / 1 from individual classifiers
  * Meta-classifier is applied on top of the individual classifiers.
  
  
* **Method of application of meta-classifiers on probabilities**:
  * Probabilities are obtained from individual classifiers.
  * Applying meta-classifier
  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("../data/WA_Fn-UseC_-HR-Employee-Attrition.csv")
num_col = list(df.describe().columns)
col_categorical = list(set(df.columns).difference(num_col))
remove_list = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
col_numerical = [e for e in num_col if e not in remove_list]
attrition_to_num = {'Yes': 0,
                    'No': 1}
df['Attrition_num'] = df['Attrition'].map(attrition_to_num)
col_categorical.remove('Attrition')
df_cat = pd.get_dummies(df[col_categorical])
X = pd.concat([df[col_numerical], df_cat], axis=1)
y = df['Attrition_num']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2)

In [4]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    '''
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

    elif train==False:
        '''
        test performance
        '''
        print("Test Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train),
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))

    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))


## Model 1: Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [9]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       189
           1       1.00      1.00      1.00       987

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[189   0]
 [  0 987]]

ROC AUC: 1.0000



In [10]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.7891

Classification Report: 
               precision    recall  f1-score   support

           0       0.36      0.38      0.37        48
           1       0.88      0.87      0.87       246

    accuracy                           0.79       294
   macro avg       0.62      0.62      0.62       294
weighted avg       0.79      0.79      0.79       294


Confusion Matrix: 
 [[ 18  30]
 [ 32 214]]

ROC AUC: 0.6225



## Model 2: Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [13]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       189
           1       1.00      1.00      1.00       987

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[189   0]
 [  0 987]]

ROC AUC: 1.0000



In [14]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8776

Classification Report: 
               precision    recall  f1-score   support

           0       0.93      0.27      0.42        48
           1       0.88      1.00      0.93       246

    accuracy                           0.88       294
   macro avg       0.90      0.63      0.68       294
weighted avg       0.88      0.88      0.85       294


Confusion Matrix: 
 [[ 13  35]
 [  1 245]]

ROC AUC: 0.6334



In [15]:
en_en = pd.DataFrame()

In [16]:
tree_clf.predict_proba(X_train)

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [17]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [18]:
en_en.head()

,tree_clf,rf_clf,Attrition_num
0,1.0,0.95,1
1,0.0,0.30,0
2,1.0,0.96,1
3,1.0,0.86,1
4,1.0,0.97,1


In [19]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

In [20]:
en_en.head()

,tree_clf,rf_clf,ind
0,1.0,0.95,1
1,0.0,0.30,0
2,1.0,0.96,1
3,1.0,0.86,1
4,1.0,0.97,1


# Meta Classifier

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
m_clf = LogisticRegression(fit_intercept=False, solver='lbfgs')

In [23]:
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

LogisticRegression(fit_intercept=False)

In [24]:
en_test = pd.DataFrame()

In [25]:
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

In [26]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

In [27]:
tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [28]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [29]:
en_test.columns = tmp

In [30]:
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined   0    1
ind              
0         18   30
1         32  214


In [31]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.7891


In [32]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.36      0.38      0.37        48
           1       0.88      0.87      0.87       246

    accuracy                           0.79       294
   macro avg       0.62      0.62      0.62       294
weighted avg       0.79      0.79      0.79       294



***

# Single Classifier

In [33]:
df = pd.read_csv("../data/WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [34]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [35]:
df.Attrition.value_counts() / df.Attrition.count()

Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: float64

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
from sklearn.ensemble import BaggingClassifier

In [38]:
from sklearn.ensemble import AdaBoostClassifier

In [39]:
class_weight = {0:0.839, 1:0.161}

In [40]:
pd.Series(list(y_train)).value_counts() / pd.Series(list(y_train)).count()

1    0.839286
0    0.160714
Name: count, dtype: float64

In [41]:
forest = RandomForestClassifier(class_weight=class_weight, n_estimators=100)

In [48]:
ada = AdaBoostClassifier(algorithm='SAMME', estimator=forest, n_estimators=100,
                         learning_rate=0.5, random_state=42)

In [49]:
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=RandomForestClassifier(class_weight={0: 0.839,
                                                                  1: 0.161}),
                   learning_rate=0.5, n_estimators=100, random_state=42)

In [50]:
print_score(ada, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       189
           1       1.00      1.00      1.00       987

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[189   0]
 [  0 987]]

ROC AUC: 1.0000



In [51]:

print_score(ada, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8673

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.19      0.32        48
           1       0.86      1.00      0.93       246

    accuracy                           0.87       294
   macro avg       0.93      0.59      0.62       294
weighted avg       0.89      0.87      0.83       294


Confusion Matrix: 
 [[  9  39]
 [  0 246]]

ROC AUC: 0.5938



In [53]:
bag_clf = BaggingClassifier(estimator=ada, n_estimators=50,
                            max_samples=1.0, max_features=1.0, bootstrap=True,
                            bootstrap_features=False, n_jobs=-1,
                            random_state=42)

In [54]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=AdaBoostClassifier(algorithm='SAMME',
                                               estimator=RandomForestClassifier(class_weight={0: 0.839,
                                                                                              1: 0.161}),
                                               learning_rate=0.5,
                                               n_estimators=100,
                                               random_state=42),
                  n_estimators=50, n_jobs=-1, random_state=42)

In [55]:
print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 0.9940

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       189
           1       0.99      1.00      1.00       987

    accuracy                           0.99      1176
   macro avg       1.00      0.98      0.99      1176
weighted avg       0.99      0.99      0.99      1176


Confusion Matrix: 
 [[182   7]
 [  0 987]]

ROC AUC: 0.9815



In [56]:
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8605

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.15      0.25        48
           1       0.86      1.00      0.92       246

    accuracy                           0.86       294
   macro avg       0.93      0.57      0.59       294
weighted avg       0.88      0.86      0.81       294


Confusion Matrix: 
 [[  7  41]
 [  0 246]]

ROC AUC: 0.5729



***